# importing functions

In [1]:
import random
from itertools import count
from matplotlib.animation import FuncAnimation
import time
import matplotlib.pyplot as plt
import follow_road



# Graphical functions

In [57]:
def draw_trajectory(traj) :
    #fig=plt.figure()
    #ax1=fig.add_subplot(111)
    #ax2=fig.add_subplot(221)
    xs, ys = zip(*traj)
    plt.plot(xs,ys)
    #ax1.set_aspect('equal')
    return

Unchecked Starred(value=Name(id='traj', ctx=Load()), ctx=Load())


In [58]:
def draw_thymio(pose) :
    move=pose[0:2]
    angle=pose[2]-math.pi/2
    thymio_shape=[[-0.5,-0.5],[0.5,-0.5],[0.5,0.25],[0,0.75],[-0.5,0.25]]
    
    thymio_shape.append(thymio_shape[0]) #repeat the first point to create a 'closed loop'
    rot=np.array([[np.cos(angle) , -np.sin(angle)],[np.sin(angle),np.cos(angle)]])
    thymio_shape=(np.transpose((np.matmul(rot,np.transpose(100*np.array(thymio_shape))))))
    thymio_shape+=[move,move,move,move,move,move]
    xs, ys = zip(*thymio_shape) #create lists of x and y values
    fig=plt.figure(figsize=(8, 8), dpi=80)

    ax = fig.add_subplot(111)
    ax.set_aspect('equal')

    plt.xlim((0,1600))
    plt.ylim((0,1100))
    plt.plot(xs,ys)
    return thymio_shape

Unchecked Starred(value=Name(id='thymio_shape', ctx=Load()), ctx=Load())


In [59]:
def draw_now(pose,traj) :
    draw_thymio(pose)
    draw_trajectory(traj)
    plt.show()
    return


# PID controller

In [111]:
def PID(e,ediff,eint) :

    Kp=12
    Ki=0.05
    Kd=0
    correction=Kp*e+Ki*eint+Kd*ediff
    return correction

def theta_des(pos,obj) :
    '''
    orientation desired to face towards the goal
    
    '''
    
       
    return math.atan2(obj[1]-pos[1],obj[0]-pos[0])

def speed_des(pos,obj) :
    '''
    straight line speed desired, constant component + variable 
    component depending on distance to next point
    '''
    vbase=500
    coeff=0.5 # weight of variable component
    return vbase+coeff*math.sqrt((obj[1]-pos[1])**2+(obj[0]-pos[0])**2)

def wheel_inputs(omega,v) :
    '''
    calculates the motor speeds for the robot
    
    Arguments:
        - omega: correction, output of the PID
        - v: desired straight line speed
    Outputs:
        - [vl,vr]: left and right motor speeds 
    
    '''
    R=20 #wheel radius in [mm]
    L=95 #distance between wheels in [mm]
    vl=5*(2*v-omega*L)/(2*R) #left motor speed
    vr=5*(2*v+omega*L)/(2*R) #right motor speed
    
    #### setting lower and upper bounds ####
    if vr<0 :
        vr=max(-300,vr)
    else :
        vr=min(300,vr)
        
    if vl<0 :
        vl=max(-300,vl)
    else :
        vl=min(300,vl)

    
    return [int(vl),int(vr)]

def compute_error(pose,obj) : 
    angle=theta_des(pose,obj)
    error=angle-pose[2]
    if error>math.pi : 
        error-=2*math.pi
    if error< -(math.pi) :
        error+=2*math.pi
    return error

In [53]:
e_diff = 0
e_int = 0
prev_error = 0
def follow_path(pose,next_point,reset,dt):
    
    max_error = 5
    global e_diff, e_int, prev_error
    if reset:
        nx_pt_id=0 # index of the next point to attain in the trajectory, set as 0 to start    
        prev_error=0 
        e_int=0
    
    
        
    ########    call pid   ##########
    
    error=compute_error(pose,next_point)
    e_diff=(error-prev_error)/dt
    
    #print("error")
    #print(error)
    #print("e_diff")
    #print(e_diff)
    
    e_int+=error
    
    if e_int > max_error:
        e_int = max_error
    if e_int < -max_error:
        e_int = -max_error
        
    #print("e_int",e_int)

    
    omega=PID(error,e_diff,e_int)
    prev_error = error
    #######  set wheel speeds    #######
    v=speed_des(pose,next_point)
    [vl,vr]=wheel_inputs(omega,v)
    #print("speeds")
    #print([vl,vr])
      
    return [vl,vr]